<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/class/egor-Homework_15_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3

conn = sqlite3.connect("shop.db")
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    date_created DATE DEFAULT (DATE('now'))
)
""")


# foreign key means in a child table when is the parent


cur.execute("""
CREATE TABLE IF NOT EXISTS orders (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    price REAL,
    date_created DATE DEFAULT (DATE('now')),
    FOREIGN KEY (user_id) REFERENCES users(id)
)
""")

conn.commit()

#Referential Integrity


Database Primary Key

parent:
none

children:
George, Yeremii<-----orphans


corrupt


commit means write changes to the database

1.  add customer

2.  start to add order (boom.  computer crashes)

3.  commit





In [2]:
cur.executemany("""
INSERT INTO users (name)
VALUES (?)
""", [
    ("Alex",),
    ("Maria",),
    ("Ivan",)
])


# here the best practice would be to do select(user_id) from customers where
# name = Alex

# nested query


cur.executemany("""
INSERT INTO orders (user_id, price)
VALUES (?, ?)
""", [
    (1, 120.50),
    (1, 80.00),
    (2, 200.00),
    (3, 50.00),
    (3, 70.00),
])

conn.commit()

In [4]:
# 1. Insert the users (same as before)
cur.executemany("""
INSERT INTO users (name)
VALUES (?)
""", [
    ("Alex",),
    ("Maria",),
    ("Ivan",)
])

# 2. Insert orders using a nested query to look up the ID by name
cur.executemany("""
INSERT INTO orders (user_id, price)
VALUES (
    (SELECT id FROM users WHERE name = ?),
    ?
)
""", [
    ("Alex", 4120.50),
    ("Alex", 480.00),
    ("Maria", 4200.00),
    ("Ivan", 450.00),
    ("Ivan", 470.00),
])

conn.commit()

In [12]:
cur.execute("""
SELECT * from orders;

""")

for row in cur.fetchall():
    print(row)

cur.execute("""
select * from users;
""")

for row in cur.fetchall():
    print(row)

(1, 1, 120.5, '2025-12-16')
(2, 1, 80.0, '2025-12-16')
(3, 2, 200.0, '2025-12-16')
(4, 3, 50.0, '2025-12-16')
(5, 3, 70.0, '2025-12-16')
(6, 1, 120.5, '2025-12-16')
(7, 1, 80.0, '2025-12-16')
(8, 2, 200.0, '2025-12-16')
(9, 3, 50.0, '2025-12-16')
(10, 3, 70.0, '2025-12-16')
(11, 1, 4120.5, '2025-12-16')
(12, 1, 480.0, '2025-12-16')
(13, 2, 4200.0, '2025-12-16')
(14, 3, 450.0, '2025-12-16')
(15, 3, 470.0, '2025-12-16')
(1, 'Alex', '2025-12-16')
(2, 'Maria', '2025-12-16')
(3, 'Ivan', '2025-12-16')
(4, 'Alex', '2025-12-16')
(5, 'Maria', '2025-12-16')
(6, 'Ivan', '2025-12-16')
(7, 'Alex', '2025-12-16')
(8, 'Maria', '2025-12-16')
(9, 'Ivan', '2025-12-16')


In [13]:

name = '''
  Walker Rowe the person who
  lives on
  this street
'''


cur.execute("""
SELECT users.name, orders.price, orders.date_created, users.id, orders.id
FROM users
JOIN orders ON users.id = orders.user_id
WHERE users.name = "Alex"
""")

for row in cur.fetchall():
    print(row)

('Alex', 120.5, '2025-12-16', 1, 1)
('Alex', 80.0, '2025-12-16', 1, 2)
('Alex', 120.5, '2025-12-16', 1, 6)
('Alex', 80.0, '2025-12-16', 1, 7)
('Alex', 4120.5, '2025-12-16', 1, 11)
('Alex', 480.0, '2025-12-16', 1, 12)


In [ ]:
# here we count


cur.execute("""
SELECT users.name, COUNT(orders.id) AS orders_count
FROM users
LEFT JOIN orders ON users.id = orders.user_id
GROUP BY users.id
""")

for row in cur.fetchall():
    print(row)

('Alex', 2)
('Maria', 1)
('Ivan', 2)


In [17]:
cur.execute("""
SELECT users.name, COUNT(orders.price) AS orders_count
FROM users
LEFT JOIN orders ON users.id = orders.user_id
GROUP BY users.id
""")

for row in cur.fetchall():
    print(row)



('Alex', 6)
('Maria', 3)
('Ivan', 6)
('Alex', 0)
('Maria', 0)
('Ivan', 0)
('Alex', 0)
('Maria', 0)
('Ivan', 0)


In [18]:
cur.execute("""
SELECT count (*) from orders
""")

for row in cur.fetchall():
    print(row)

(15,)


In [35]:
# cursor.

cur.execute("""
SELECT users.name, SUM(orders.price) AS total_sum
FROM users
JOIN orders ON users.id = orders.user_id
GROUP BY users.id
""")

col_names = [desc[0] for desc in cur.description]
print(col_names)


for row in cur.fetchall():
    print(row)

['name', 'total_sum']
('Alex', 7006.5)
('Maria', 6600.0)
('Ivan', 2360.0)


In [28]:
cur.executemany("""
INSERT INTO orders (user_id, price)
VALUES (?, ?)
""", [
    (1, 120.50),
    (1, 80.00),
    (2, 200.00),
    (3, 50.00),
    (3, 70.00),
])

conn.commit()

# Homework

1.  make sure you understand all of this

2.  Extra Credit: explain how this random function works

In [47]:
cur.execute("""
INSERT INTO orders (price, date_created)
VALUES (
  -- random real 12–250; wrap in CAST(...) AS INT if you want integer
  12 + (abs(random()) % (250 - 12 + 1)),

  -- random date in last 500 days
  date('now', printf('-%d days', abs(random()) % 500))
);
""")

conn.commit()

In [41]:
cur.execute("""
 select strftime('%Y', date_created) AS year,
        strftime('%m', date_created) AS month,
        strftime('%d', date_created) AS day
 from orders
""")

for row in cur.fetchall():
    print(row)

('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12', '16')
('2025', '12'

In [48]:
cur.execute("""
 select strftime('%Y', date_created) AS year,
        strftime('%m', date_created) AS month,
        SUM(orders.price) AS total_sum
from orders
group by year, month
order by year, month
""")

for row in cur.fetchall():
    print(row)

('2024', '08', 74.0)
('2024', '11', 44.0)
('2024', '12', 14.0)
('2025', '05', 39.0)
('2025', '10', 135.0)
('2025', '12', 15966.5)
